<img src="http://s3.amazonaws.com/pix.iemoji.com/images/emoji/apple/ios-12/256/waving-hand.png" align=left width=44, heigth=44>
<div class="alert alert-info">
<b> Комментарий ревьюера</b>


Привет, Сергей! Давай знакомиться! Меня зовут Дмитрий Махортов, и я буду проверять твой проект. Сразу предлагаю общение на «ты» 🙂, но если тебе это не комфортно, то дай знать, и мы перейдем на «вы». 

Моя основная цель — не указать на совершенные тобою ошибки, а поделиться своим опытом и помочь тебе погрузиться в увлекательный мир работы с данными и вырасти в крепкого профи. Это отдаленная цель. А ближайшая - сделать твою работу еще лучше )).
   
    
Все ключевые этапы в работе выполнены, и я вижу что с проектом ты справшяешься. Есть моменты, которые нужно доработать, но я уверен, у тебя все получится.
        
Расскажу как обычно проходит проверка:    
Бывают моменты, которые требуют пристального внимания. Комментарии по ним выделены <span style='background-color:#F7B3A4'> красным цветом </span> и обозначены значком 🛑. После их доработки проект будет принят.  🙂
 
<span style='background-color:#B7EBA7'> Зеленым цветом </span> и значком ✅ отмечены удачные и элегантные решения, на которые можно опираться в будущих проектах. Или советы «со звездочкой», которые помогут тебе в будущем.

<span style='background-color:#F9EDA6'>Жёлтым цветом </span> и значком ⚠️ выделено то, что в следующий раз можно сделать по-другому. Ты можешь учесть эти комментарии при выполнении будущих заданий или доработать проект сейчас (однако это не обязательно).

Давай работать над проектом в диалоге: **если ты что-то меняешь в проекте по моим рекомендациям — пиши об этом**. Выбери для своих комментариев какой-то заметный цвет, так мне будет легче отследить изменения. Пожалуйста, **не перемещай, не изменяй и не удаляй мои комментарии**. Всё это поможет выполнить повторную проверку твоего проекта оперативнее. 
</div>

<div class="alert" style="background-color:#ead7f7;color:#8737bf">
    <font size="3"><b>образец комментария студента</b></font>
   
Можешь использовать этот комментарий. Просто удали этот текст и вместо него напиши свой 🙂

</div>

# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

### Описание данных

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. 

Известно:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
-  is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

### Импортируем требуемые для работы библиотеки

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier

import sklearn.metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

<img src="https://emojigraph.org/media/apple/check-mark-button_2705.png" align=left width=33, heigth=33>
<div class="alert alert-success">
Отлично, все нужные библиотеки импортированы в начале ноутбука.Это хорошая практика.</div>

### Загружаем файл

In [2]:
try:
    df = pd.read_csv('/datasets/users_behavior.csv')
except:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv')

### Ознакамливаемся с данными

In [3]:
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


### Смотрим тип данных и пропуски

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


<img src="https://emojigraph.org/media/apple/check-mark-button_2705.png" align=left width=33, heigth=33>
<div class="alert alert-success">
Данные загружены и исследованы.</div>

## Разбейте данные на выборки

### Разбиваем данные на признаки и целевой признак

In [5]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

### Проверяем полученные таблицы

In [6]:
features.head()

,calls,minutes,messages,mb_used
0,40.0,311.90,83.0,19915.42
1,85.0,516.75,56.0,22696.96
2,77.0,467.66,86.0,21060.45
3,106.0,745.53,81.0,8437.39
4,66.0,418.74,1.0,14502.75


In [7]:
target.head()

0    0
1    0
2    0
3    1
4    0
Name: is_ultra, dtype: int64

### Разделяем данные на выборки

In [8]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=.25, random_state=12345)

In [9]:
features_test, features_val, target_test, target_val = train_test_split(
    features_test, target_test, test_size=.5, random_state=12345)

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/81/Stop_sign.png/240px-Stop_sign.png" align=left width=35, heigth=35>
<div class="alert alert-danger">
<b>v2</b>    
Долго не мог понять, почему у тебя получены такие хорошие результаты, особенно на тестировании.  И причина в том, что разделение на выборки произведено с ошибкой.
    
Данных у нас всего 3214, а размеры выборок 2410 + 1607 + 1607, т.е. очевидно, что выборки пересекаются. А значит мы имеем утечки из валидации и теста в обучающую выборку.  Т.е. многие данные, на которых мы тестируем модель уже увидела в процессе обучения.  
    
<div class="alert" style="background-color:#ead7f7;color:#8737bf">
   
<b>v2</b> Исправил:) Прошу прощение за невнимарельность</div>
    
<div class="alert alert-success">
<b>v3</b> исправлено </div>    
</div>

<img src="https://emojigraph.org/media/apple/check-mark-button_2705.png" align=left width=33, heigth=33>
<div class="alert alert-success">
Выборки получены корректно</div>

<img src="https://emojigraph.org/media/apple/check-mark-button_2705.png" align=left width=33, heigth=33>
<div class="alert alert-success">
Совет:  обрати внимание на аргумент stratify функции train_test_split. Стратификация особенно важна, когда у нас мало данных и возможен дисбаланс классов в обучении / валидации / тесте.
</div>

<img src="https://upload.wikimedia.org/wikipedia/commons/b/ba/Warning_sign_4.0.png" align=left width=44, heigth=33>
<div class="alert alert-warning">
Рекомендую явно вывести размеры получившихся выборок (с использованием .shape и print), это поможет быстро оценить корректность разделения на выборки.
    
<div class="alert alert-success">
<b>v2</b> исправлено </div>    
</div>

In [24]:
print(features_train.shape)
print(features_test.shape)
print(features_val.shape)
print(target_train.shape)
print(target_test.shape)
print(target_val.shape)

(2410, 4)
(402, 4)
(402, 4)
(2410,)
(402,)
(402,)


<div class="alert" style="background-color:#ead7f7;color:#8737bf">
   
Добавил:)

</div>

Получаем следующие выборки: обучающая (75%), валидационная (12.5%) и тестовая (12.5%).

## Исследуйте модели

### Решающее дерево

In [11]:
best_tree_model = None
best_tree_result = 0
best_tree_depth = 0
for tree_depth in range(1, 4):
    tree_model = DecisionTreeClassifier(random_state=12345, max_depth=tree_depth)
    tree_model.fit(features_train, target_train)
    tree_predictions_val = tree_model.predict(features_val)
    tree_result = accuracy_score(target_val, tree_predictions_val)
    if tree_result > best_tree_result:
        best_tree_model = tree_model
        best_tree_result = tree_result
        best_tree_depth = tree_depth
print("Accuracy наилучшей модели на валидационной выборке:", best_tree_result, "Глубина дерева:", best_tree_depth)

Accuracy наилучшей модели на валидационной выборке: 0.7786069651741293 Глубина дерева: 3


<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/81/Stop_sign.png/240px-Stop_sign.png" align=left width=35, heigth=35>
<div class="alert alert-danger">
Мы решаем задачу классификации, а модель DecisionTreeRegressor предназначена для решения задачи регрессии. Пожалуйста используй правильную модель.

<div class="alert alert-success">
<b>v2</b> исправлено </div>

</div>

<div class="alert" style="background-color:#ead7f7;color:#8737bf">
   
Поправил:)

</div>

<img src="https://emojigraph.org/media/apple/check-mark-button_2705.png" align=left width=33, heigth=33>
<div class="alert alert-success"><b>v2</b>
Дерево решений исследовано  корректно:
    
 - исследовано 3 значения гиперпараметров
 - модель обучена на обучающем наборе
 - получена оценка качества на валидационном наборе
 - перебор гиперпараметров осуществляется в цикле
</div>

### Случайный лес

In [12]:
best_forest_model = None
best_forest_result = 0
best_forest_est = 0
best_forest_depth = 0
for forest_est in range(10, 51, 10):
    for forest_depth in range (1, 11):
        forest_model = RandomForestClassifier(random_state=12345, n_estimators=forest_est, max_depth=forest_depth)
        forest_model.fit(features_train, target_train)
        forest_predictions_val = forest_model.predict(features_val)
        forest_result = accuracy_score(target_val, forest_predictions_val)
        if forest_result > best_forest_result:
            best_forest_model = forest_model
            best_forest_result = forest_result
            best_forest_est = forest_est
            best_forest_depth = forest_depth

print("Accuracy наилучшей модели на валидационной выборке:", best_forest_result, 
      "Количество деревьев:", best_forest_est, "Максимальная глубина:", best_forest_depth)

Accuracy наилучшей модели на валидационной выборке: 0.8109452736318408 Количество деревьев: 30 Максимальная глубина: 9


<img src="https://emojigraph.org/media/apple/check-mark-button_2705.png" align=left width=33, heigth=33>
<div class="alert alert-success"><b>v2</b>
Случайный лес исследован корректно:
    
 - исследовано 5х10  комбинаций значений гиперпараметров
 - модель обучена на обучающем наборе
 - получена оценка качества на валидационном наборе
 - перебор гиперпараметров осуществляется в цикле
</div>

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/81/Stop_sign.png/240px-Stop_sign.png" align=left width=35, heigth=35>
<div class="alert alert-danger"><b>v2</b>
Обрати внимание, что ты выводишь не лушую найденную глубину (best_forest_depth), а последнее значение глубины которое мы рассмотрели в цикле (forest_depth).
<div class="alert" style="background-color:#ead7f7;color:#8737bf">
   
<b>v2</b> Исправил:) Прошу прощение за невнимарельность 2.0</div>
<div class="alert alert-success">
<b>v3</b> исправлено </div>    
    
</div>

<img src="https://upload.wikimedia.org/wikipedia/commons/b/ba/Warning_sign_4.0.png" align=left width=44, heigth=33>
<div class="alert alert-warning"><b>v2</b>
Старайся не делать такие длинные строки. Стандарт PEP8 регулирует длину строки 79 символов. Придерживаться такого очень сложно, но лучше хотя-бы вмещать в экран, чтобы не приходилось использовать scroll.
    
    
А еще  если сомневаешься, как отформатировать правильно - всегда можно найти какой-нибудь оналйн-форматтер, например этот
    
  https://extendsclass.com/python-formatter.html
    
    
<div class="alert" style="background-color:#ead7f7;color:#8737bf">
   
<b>v2</b> Поправил:)</div>
</div>

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/81/Stop_sign.png/240px-Stop_sign.png" align=left width=35, heigth=35>
<div class="alert alert-danger">
Мы решаем задачу классификации, а модель DecisionTreeRegressor предназначена для решения задачи регрессии. Пожалуйста используй правильную модель.

<div class="alert alert-success">
<b>v2</b> исправлено </div>
</div>

<div class="alert" style="background-color:#ead7f7;color:#8737bf">
   
Поправил:)

</div>

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/81/Stop_sign.png/240px-Stop_sign.png" align=left width=35, heigth=35>
<div class="alert alert-danger">
Закончен важный этап: исследованы модели, найдены оптимальные гиперпараметры для каждой. Нужно зафиксировать результат: выбрать лучшую модель (включая гиперпараметры)  и описать свое решение в выводах.

<div class="alert alert-success">
<b>v2</b> исправлено </div>
</div>

<div class="alert" style="background-color:#ead7f7;color:#8737bf">
   
Добавил:)

</div>

- Была исследована эффективность 2-х моделей: решающее дерево и случайный лес.
- Для решающего дерева были проанализированы глубины с 1 до 5, глубина 3 имела максимальный accuracy.
- Для случайного леса были проанализированы десятки деревьев от 10 до 50 и глубины с 1 до 10. Самым эффективным оказалось 30 деревьев с максимальной глубиной равной 9.
- Из двух моделей случайный лес показал большую эффективность, accuracy равнялся ~81%.

## Проверьте модель на тестовой выборке

In [13]:
final_forest_model = RandomForestClassifier(random_state=12345, n_estimators=30, max_depth=9)
final_forest_model.fit (features_train, target_train)
final_forest_prediction = final_forest_model.predict(features_test)
final_forest_result = accuracy_score(target_test, final_forest_prediction)
print('Accuracy наилучшей модели на тестовой выборке:', final_forest_result)

Accuracy наилучшей модели на тестовой выборке: 0.8084577114427861


<img src="https://emojigraph.org/media/apple/check-mark-button_2705.png" align=left width=33, heigth=33>
<div class="alert alert-success"><b>v2</b>
Молодец: 
    
  - выбрана лучшая модель
  - тестирование произведено корректно 
  - достигнута хорошая точность на тестовой выборке
</div>


<img src="https://emojigraph.org/media/apple/check-mark-button_2705.png" align=left width=33, heigth=33>
<div class="alert alert-success"><b>v2</b>
Совет:  Сейчас, когда гиперпараметры подобраны и валидационная выборка нам не нужна, мы можем обучить модель на большой выборке (обучающая + валидационная), а тестирование новой модели произвести уже на тестовой выборке.
    
Объединить выборки можно с помощью pd.concat
</div>

## (бонус) Проверьте модели на адекватность

<img src="https://upload.wikimedia.org/wikipedia/commons/b/ba/Warning_sign_4.0.png" align=left width=44, heigth=33>
<div class="alert alert-warning">
Ты провел интересное исследование, но проверка на адекватность это чуть чуть другое.    
    
В основе проверки на адекватность лежит очень простая идея. Модель считается адекватной, если способна "побить" любой "наивный" алгоритм, например предсказывающий случайный или постояный ответ. 

Для нашего проекта самую высокую точность дает  "наивный" алгоритм, всегда предсказывающий самый популярный ответ (у нас это 0). Проверить точность такого подхода можно вручную или используя DummyClassifier.
    
<div class="alert alert-success">
<b>v2</b> исправлено </div>    
</div>

<div class="alert" style="background-color:#ead7f7;color:#8737bf">
   
Добавил:)

</div>

### Проверим модель на адекватность при сравнении с DummyClassifier

In [14]:
dummy_clf = DummyClassifier(random_state=12345, strategy="most_frequent")
dummy_clf.fit(features_train, target_train)
dummy_prediction = dummy_clf.predict(features_test)
dummy_score = accuracy_score(target_test, dummy_prediction)

### Создадим матрицу ошибок для бинарной классификации

#### RandomForestClassifier

In [15]:
confusion_matrix = sklearn.metrics.confusion_matrix(target_test, final_forest_prediction)
confusion_matrix = np.flip(confusion_matrix)
confusion_matrix = pd.DataFrame(confusion_matrix, columns = ['+Positive', '-Negative'])
confusion_matrix.set_index(pd.Index(['+Positive', '-Negative']))

,+Positive,-Negative
+Positive,63,59
-Negative,18,262


In [16]:
true_positive = 63 / len(final_forest_prediction) * 100
false_negative = 59 / len(final_forest_prediction) * 100
false_positive = 18 / len(final_forest_prediction) * 100
true_negative = 262 / len(final_forest_prediction) * 100

print(true_positive, false_negative, false_positive, true_negative)

15.671641791044777 14.676616915422885 4.477611940298507 65.17412935323384


64 раз, ~16% модель правильно классифицировала Positive как Positive.

58 раз, ~14% модель неправильно классифицировала Positive как Negative.

18 раза, ~5% модель неправильно классифицировала Negative как Positive.

262 раз, ~65% модель правильно классифицировала Negative как Negative.

#### DummyClassifier

In [17]:
confusion_matrix = sklearn.metrics.confusion_matrix(target_test, dummy_prediction)
confusion_matrix = np.flip(confusion_matrix)
confusion_matrix = pd.DataFrame(confusion_matrix, columns = ['+Positive', '-Negative'])
confusion_matrix.set_index(pd.Index(['+Positive', '-Negative']))

,+Positive,-Negative
+Positive,0,122
-Negative,0,280


Видим, что DummyClassifier фиксирует все ответы, аналогично самым популярным, в нашем случае - 0 или негативными.

### Проверим Accuracy

#### RandomForestClassifier

In [18]:
final_forest_result

0.8084577114427861

Модель показывает ~81% точности предсказаний.

#### DummyClassifier

In [19]:
dummy_score

0.6965174129353234

DummyClassifier показывает ~70% точности предсказаний.

### Проверим Precision

#### RandomForestClassifier

In [20]:
precision = sklearn.metrics.precision_score(target_test, final_forest_prediction)
print(precision)

0.7777777777777778


Модель показывает ~78% точности положительных предсказаний.
Если модель называет целевой признак положительным, то с вероятностью ~78% это так и есть.

#### DummyClassifier

In [21]:
dummy_precision = sklearn.metrics.precision_score(target_test, dummy_prediction)
print(dummy_precision)

0.0


/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Проверим Recall

#### RandomForestClassifier

In [22]:
recall = sklearn.metrics.recall_score(target_test, final_forest_prediction)
print(recall)

0.5163934426229508


Модель показывает ~52% точности при поиске положительных значений.
Модель находит лишь ~52% всех положительных значений. Зная это, мы можем понять, что предсказания по количеству переходов на новый тариф будут ниже, чем фактические показатели.

#### DummyClassifier

In [23]:
dummy_recall = sklearn.metrics.recall_score(target_test, dummy_prediction)
print(dummy_recall)

0.0


# Выводы
- Была исследована эффективность 2-х моделей: решающее дерево и случайный лес.
- Для решающего дерева были проанализированы глубины с 1 до 5, глубина 3 имела максимальный accuracy.
- Для случайного леса были проанализированы десятки деревьев от 10 до 50 и глубины с 1 до 10. Самым эффективным оказалось 30 деревьев с максимальной глубиной равной 9.
- Из двух моделей случайный лес показал большую эффективность, accuracy равнялся ~81%.

Данная модель имела следующие показатели:
- 64 раз, ~16% модель правильно классифицировала Positive как Positive.
- 58 раз, ~14% модель неправильно классифицировала Positive как Negative.
- 18 раза, ~5% модель неправильно классифицировала Negative как Positive.
- 262 раз, ~65% модель правильно классифицировала Negative как Negative.
- Видим, что DummyClassifier фиксирует все ответы, аналогично самым популярным, в нашем случае - 0 или негативными.
- Модель показывает ~81% точности предсказаний.
- DummyClassifier показывает ~70% точности предсказаний.
- Модель показывает ~78% точности положительных предсказаний.
- Если модель называет целевой признак положительным, то с вероятностью ~78% это так и есть.
- Модель показывает ~52% точности при поиске положительных значений.
- Модель находит лишь ~52% всех положительных значений. Зная это, мы можем понять, что предсказания по количеству переходов на новый тариф будут ниже, чем фактические показатели.

<img src="https://emojigraph.org/media/apple/check-mark-button_2705.png" align=left width=33, heigth=33>
<div class="alert alert-success">
 👍 </div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75

<img src="http://s3.amazonaws.com/pix.iemoji.com/images/emoji/apple/ios-12/256/waving-hand.png" align=left width=44, heigth=44>
<div class="alert alert-info">
<b> Заключительный комментарий</b>

К сожалению ты использовал модели регрессии вместо моделей классификации. Пожалуйста исправь ошибку, после чего я смогу закончить ревью.
    
Со своей стороны постарался проверить и прокомментировать по максимуму.    
</div>

<img src="http://s3.amazonaws.com/pix.iemoji.com/images/emoji/apple/ios-12/256/waving-hand.png" align=left width=44, heigth=44>
<div class="alert alert-info">
<b> Заключительный комментарий v2</b>

Продолжаем ревью.

В целом проект уже почти готов, но нужно исправить две ошибки
    
 - Во первых выборки, на которые ты разделил наш датасет  не должны пересекаться
 - А во вторых, вывододя информацию о случайном лесе нужно вывести лучшую глубину, а не последнее значение глубины, которое мы исследовали в цикле.
    
<div class="alert" style="background-color:#ead7f7;color:#8737bf">
   
<b>v2</b> Все предварительные выводы и финальный вывод поправил. Заметил, что и в модель для тестирования не добавил лучшие гиперпараметры, сейчас все корректрно.</div>

</div>

<img src="http://s3.amazonaws.com/pix.iemoji.com/images/emoji/apple/ios-12/256/waving-hand.png" align=left width=44, heigth=44>
<div class="alert alert-info">
<b> Заключительный комментарий v3</b>

На этот раз все ОК.    
    
Проект принят. Поздравляю и желаю дальнейших успехов!
</div>